# Map my scenes

A small tool based on the [OData catalogue](https://documentation.dataspace.copernicus.eu/APIs/OData.html) to map and download the footprints of satellite scenes.


## Introduction

The [Copernicus Browser](https://browser.dataspace.copernicus.eu/) allows you to quickly and easily search the Earth Observation data available in the Copernicus Data Space Ecosystem's [Data Collections](https://dataspace.copernicus.eu/explore-data/data-collections). The search functionality displays the footprints of the products matching your criteria on a map, and allows you to browse the metadata for the individual scenes returned. If you need further functionalities, you can use the [Catalogue APIs](https://dataspace.copernicus.eu/analyse/apis/catalogue-apis) to retrieve specific metadata for selected scenes. 

But what if you simply want to see the coverage of some specific products on a map or import the footprints to a GIS interface?

This Jupyter Notebook provides a small tool built on top of the [OData catalogue](https://documentation.dataspace.copernicus.eu/APIs/OData.html) allowing you to map a single or multiple satellite scenes, as well as export the metadata as a geospatial vector file (GeoJSON, GeoPackage, Shapefile or FlatGeobuf) for use in a GIS application.

This little utility tool is simple to use, requires no account creation or authentication, and demonstrates how easy it is to build on top of the Copernicus Data Space Ecosystem services.

## Import libraries

Let's start by importing the Python libraries need for the tool to run. 

_Note: these libraries are all part of the [Geoscience Python environment](https://documentation.dataspace.copernicus.eu/Applications/JupyterHub.html#jupyterlab-user-interface), meaning no additional installation is needed when running this Jupyter Notebook in the Copernicus Data Space Ecosystem [JupyterLab](https://dataspace.copernicus.eu/analyse/jupyterlab)._

In [1]:
from pathlib import Path
from typing import List, Optional

import requests
import geopandas as gpd
from shapely.geometry import Polygon
import folium

## Tool definition

The following cell contains the tool built on the OData based catalogue. It is built as a Python class. To use the tool, simply execute this cell and move to the usage examples lower down.

In [2]:
class MapMyScenes:
    """
    Fetch the footprints for given satellite scenes.

    This class handles the odata queries for the satellite scenes
    and builds an output geospatial file.
    """

    def __init__(
        self,
        input_products: Optional[list] = None,
    ):
        if isinstance(input_products, str):
            self.input_products = [input_products]
        elif isinstance(input_products, list):
            self.input_products = input_products
        elif input_products is None:
            self.input_products = None
        else:
            raise ValueError("input_products must be a list or a string")
        self.catalog_entries = None
        self.no_data = []

    def query_catalog(self, search_type: str = "PARTIAL") -> List[dict]:
        if not self.input_products:
            raise ValueError("No input data provided.")
        if search_type not in ["PARTIAL", "EXACT"]:
            raise ValueError("Invalid search type. Choose from: ['PARTIAL', 'EXACT']")

        json_response = []
        for product in self.input_products:
            try:
                if search_type == "PARTIAL":
                    response = requests.get(
                        f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=startswith(Name, '{product}')"
                    )
                else:
                    response = requests.get(
                        f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Name eq '{product}'"
                    )
                json_data = response.json()

                if "value" not in json_data or not json_data["value"]:
                    self.no_data.append(product)
                    continue

                json_response.append(json_data["value"][0])

            except requests.exceptions.RequestException as e:
                print(f"Failed to query the catalogue for {product}: {e}")
                self.no_data.append(product)

        if json_response:
            self.catalog_entries = gpd.GeoDataFrame(json_response)
            self.catalog_entries.set_geometry(
                self.catalog_entries["GeoFootprint"].apply(
                    lambda x: Polygon(x["coordinates"][0]) if x else None
                ),
                inplace=True,
            )
            self.catalog_entries.set_crs(epsg=4326, inplace=True)

    def plot_footprints(self, label: str = None) -> folium.Map:
        if self.catalog_entries is None:
            raise ValueError("No catalog entries found.")
        reprojected_gdf = self.catalog_entries.to_crs(epsg=4326)
        m = folium.Map()
        fit_bounds = reprojected_gdf.total_bounds.tolist()
        m.fit_bounds([[fit_bounds[1], fit_bounds[0]], [fit_bounds[3], fit_bounds[2]]])
        if label:
            folium.GeoJson(
                reprojected_gdf,
                popup=folium.GeoJsonPopup(
                    fields=["Name"],
                    max_width=800,
                    style="white-space: normal; word-break: break-all;",  # Make text wrap
                ),
            ).add_to(m)
        else:
            folium.GeoJson(reprojected_gdf).add_to(m)

        return m

    def download_footprints(
        self, filename: str, output_path: str, format: str = "gpkg"
    ) -> None:
        if self.catalog_entries is None:
            raise ValueError("No catalog entries found.")

        valid_formats = {
            "gpkg": "GPKG",
            "shp": "ESRI Shapefile",
            "geojson": "GeoJSON",
            "fgb": "FlatGeobuf",
        }

        if format not in valid_formats.keys():
            raise ValueError(f"Invalid format. Choose from: {valid_formats.keys()}")

        output_dir = Path(output_path)
        output_dir.mkdir(parents=True, exist_ok=True)
        outpath = output_dir / f"{filename}.{format}"

        self.catalog_entries.to_file(outpath, driver=valid_formats[format])

## Examples

### 1. Mapping a single scene.

In this first example, we will map the extent of a single Sentinel-2 scene.

Let's start by specifying the scene name.

_Note: If you want to search for the full name of the product (much faster search), you need to specify the `.SAFE` extension in the product name. You can search without the extension, as demonstrated further down, but the search will be slower._

In [3]:
s2_single_scene = "S2B_MSIL1C_20250213T100029_N0511_R122_T33TVN_20250213T115442.SAFE"

We now initialise the tool that was defined further up in this Notebook and pass the scene name as a parameter. We can then search the OData catalogue for the exact product name (requires `.SAFE` extension in the product name for Sentinel-2).

In [4]:
# Initialise the class with the input data
my_scene = MapMyScenes(s2_single_scene)

# Search the catalogue
my_scene.query_catalog(search_type="EXACT")

We are now able to plot the scene using the `plot_footprints` function.

In [5]:
# Plot the footprints and display on a map
map = my_scene.plot_footprints()
display(map)

### 2. Mapping multiple scenes and downloading information

In this second example, we will run the same commands as we did for a single product, but will pass a list of two Sentinel-2 products to the tool. In a following step, we will download the results locally for use in a GIS application.

In [6]:
# Specify a list of Sentinel-2 scenes
s2_multiple_scenes = [
    "S2B_MSIL2A_20250312T095029_N0511_R079_T33TWN_20250312T124423.SAFE",
    "S2B_MSIL2A_20250311T101739_N0511_R065_T32TNS_20250311T142040.SAFE",
]

# Initialise the class with the input data
my_scenes = MapMyScenes(s2_multiple_scenes)

# Search the catalogue
my_scenes.query_catalog(search_type="EXACT")

# Plot the footprints and display on a map
map_scenes = my_scenes.plot_footprints()
display(map_scenes)

If you want to download the footprints and their associated metadata in a format compatible with a GIS tool, you can simply call the `download_footprints` function.

The first parameter is the name of the file, the second parameter is the location on your computer where you would like to download the file, and the third parameter is the file format. Currently, the following formats are supported:

- `gpkg`, for [Geopackage](https://www.geopackage.org/)
- `shp`, for [ESRI Shapefile](https://enterprise.arcgis.com/en/portal/11.3/use/shapefiles.htm)
- `geojson`, for [GeoJSON](https://geojson.org/)
- `fgb`, for [FlatGeobuf](https://flatgeobuf.org/)

In [7]:
# Save the footprints as a GeoJSON file called 'two_scenes.geojson' in the 'Downloads' folder
my_scenes.download_footprints("two_scenes", "~/Downloads", "geojson")

### 3. Mapping multiple scenes from different sensors and downloading footprints (with Metadata)

The tool also allows you to search for multiple products in the [data catalogue](https://dataspace.copernicus.eu/explore-data/data-collections) of the Copernicus Data Space Ecosystem.

In this third example, we will search for the following products:

- Sentinel-1 [GRD](https://documentation.dataspace.copernicus.eu/Data/SentinelMissions/Sentinel1.html#sentinel-1-level-1-ground-range-detected-grd)
- Sentinel-1 [SLC](https://documentation.dataspace.copernicus.eu/Data/SentinelMissions/Sentinel1.html#sentinel-1-level-1-single-look-complex-slc)
- [Pleiades](https://documentation.dataspace.copernicus.eu/Data/Others/CCM.html#pleiades)
- [Copernicus Digital Elevation Model](https://documentation.dataspace.copernicus.eu/Data/ComplementaryData/Additional.html#copernicus-digital-elevation-model-cop-dem-europe)

As you can see on the map below, it is difficult to determine which products correspond to which footprints. To display the name of the product when clicking on a footprint, you can add the `label=True` parameter to the `plot_footprints()` function.

In [8]:
# Specify the product names for a mixed list of scenes
mixed_multiple_scenes = [
    "S1A_IW_GRDH_1SDV_20250213T045421_20250213T045446_057873_072391_69C2.SAFE",
    "S1A_IW_SLC__1SDV_20250219T165111_20250219T165139_057968_072762_EF39.SAFE",
    "PH1A_PHR_MS___3_20221007T102302_20221007T102305_TOU_1234_6206.DIMA",
    "DEM1_SAR_DGE_10_20110620T165003_20140909T165150_ADS_000000_8486_76ce6e2c.DEM",
]

# Initialise the class with the input data
my_mixed_scenes = MapMyScenes(mixed_multiple_scenes)

# Search the catalogue
my_mixed_scenes.query_catalog(search_type="EXACT")

# Plot the footprints and display on a map, adding labels
map_mixed_scenes = my_mixed_scenes.plot_footprints(label=True)
display(map_mixed_scenes)

# Save the footprints as a GeoJSON file called 'mixed_scenes.gpkg' in the 'Downloads' folder
my_mixed_scenes.download_footprints("mixed_scenes", "~/Downloads", "gpkg")

### 4. Partial search

If you are not sure of the file extension of your product or would like to perform a search with a partial product name, you can use the `search_type="PARTIAL"` option in the catalogue search. However, the search time will be significantly longer.

In this fourth and last example, we will search for the following products:

- [Sentinel-3 OLCI](https://documentation.dataspace.copernicus.eu/Data/SentinelMissions/Sentinel3.html#sentinel-3-olci-level-1)
- [Sentinel-5P](https://documentation.dataspace.copernicus.eu/Data/SentinelMissions/Sentinel5P.html)

The products' full names are:

- `S3A_OL_1_EFR____20250219T093203_20250219T093503_20250219T112913_0179_122_364_2160_PS1_O_NR_004.SEN3`
- `S5P_NRTI_L2__CLOUD__20250219T141619_20250219T142119_38110_03_020700_20250219T145111.nc`. 

In this example, we will truncate the Sentinel-3 product's name and pass the Sentinel-5P's name without the .nc extension.

In [9]:
# Specify the product names
incomplete_products = [
    "S3A_OL_1_EFR____20250219T093203_20250219T093503_",
    "S5P_NRTI_L2__CLOUD__20250219T141619_20250219T142119_38110_03_020700_20250219T145111",
]

# Initialise the class with the input data
my_incomplete_scenes = MapMyScenes(incomplete_products)

# Search the catalogue
my_incomplete_scenes.query_catalog(search_type="PARTIAL")

map_found_scenes = my_incomplete_scenes.plot_footprints(label=True)
display(map_found_scenes)